# Convert PyTorch Checkpoint to Huggingface Transformers Model

PyTorch の Checkpoint から Transformers 形式に変換するコード。

In [ ]:
# 注意点：Mixed Precision を使用している際、Transformers のバージョンを合わせないと dtype が間違って cast されることがある
!pip install transformers==4.21.0

In [ ]:
import json
import torch
import transformers
from transformers import GPTNeoXForCausalLM, AutoModelForCausalLM, AutoTokenizer
transformers.__version__

In [ ]:
# Change here
model_dir = "neox_3_6b_instruct"

In [ ]:
# Rename file (only first time)
!mv {model_dir}/fullmodel.pt {model_dir}/pytorch_model.bin

In [ ]:
# ダウンロードした重みを読み込む
user_content = torch.load(f"{model_dir}/user_content_fullmodel.pt")
full_model = torch.load(f"{model_dir}/pytorch_model.bin")

In [ ]:
# Config を取り出す
hf_config = user_content['model_config']
hf_config.to_json_file(f"{model_dir}/config.json")
hf_config

In [ ]:
# dype を確認する
for key, value in full_model.items():
    print(key, value.dtype)

## 検証

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b")
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

In [ ]:
# dtype が合っているか確認する
for key, value in model.state_dict().items():
    print(key, value.dtype)

In [ ]:
prompt_template = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"
instruction = "コロンの香りの悪いアイデアを投書してください。"
prompt = prompt_template.format(instruction=instruction)

inputs = tokenizer(
    prompt,
    add_special_tokens=False,
    return_token_type_ids=False,
    return_tensors="pt"
).to(model.device)
generation_output = model.generate(
    **inputs,
    max_new_tokens=50,
    return_dict_in_generate=True,
    output_scores=True,
    do_sample=True,
    temperature=0.3,
    pad_token_id=1,
    eos_token_id=0,
)
s = generation_output.sequences[0]
output = tokenizer.decode(s, skip_special_tokens=True)
print(output)